## Creating colour mappings for vis

Here, we use python and SPARQL to create mappings from entities to their different properties, which we can use to colour an embedding visualisation.

In [1]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import os

def make_sparql_request(query):
    sparql = SPARQLWrapper("http://63.33.68.17:3030/heritage-connector/sparql")
    sparql.setQuery(query)
    sparql.setMethod("POST")
    sparql.setReturnFormat(JSON)
    
    return sparql.query().convert()

In [2]:
ent_mapping_path = "../data/processed/final_model_dglke/entities.tsv"

ent_idx_mapping = pd.read_csv(
    ent_mapping_path,
    sep="\t",
    index_col=0,
    header=None,
    names=["value"],
).fillna("")


In [4]:
ent_idx_mapping.head()

,value
0,https://collection.sciencemuseumgroup.org.uk/p...
1,http://www.wikidata.org/entity/Q3568968
2,https://collection.sciencemuseumgroup.org.uk/o...
3,plastic
4,https://collection.sciencemuseumgroup.org.uk/p...


### 1. By database

categories: collection, journal, blog, Wikidata, V&A, 

In [5]:
def map_to_database(l):
    if 'collection.sciencemuseumgroup' in l:
        return 'SMG collection'
    elif 'blog.sciencemuseum' in l:
        return 'SMG blog'
    elif 'journal.sciencemuseum' in l:
        return 'SMG journal'
    elif 'wikidata.org/entity' in l:
        return 'Wikidata'
    else:
        return None


mapping_database = ent_idx_mapping.copy()
mapping_database['group'] = mapping_database['value'].apply(map_to_database)

mapping_database.head()

,value,group
0,https://collection.sciencemuseumgroup.org.uk/p...,SMG collection
1,http://www.wikidata.org/entity/Q3568968,Wikidata
2,https://collection.sciencemuseumgroup.org.uk/o...,SMG collection
3,plastic,None
4,https://collection.sciencemuseumgroup.org.uk/p...,SMG collection


## 2. By type (org/person/object etc)

categories: SMG person, SMG organisation, SMG object, blog, journal, Wikidata entity

In [6]:
type_query = """PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
SELECT DISTINCT ?s ?o WHERE {        
  ?s skos:hasTopConcept ?o.
}
"""

res = make_sparql_request(type_query)

In [7]:
# TODO: use the above skos:hasTopConcept results as a starting point for this instead of the blank mappings

res_df = pd.json_normalize(res['results']['bindings'])[['s.value', 'o.value']]
mapping_type = ent_idx_mapping.copy()

mapping_type = pd.merge(
    left=mapping_type, 
    right=res_df, 
    left_on="value",
    right_on="s.value",
    how="left"
)[['value', 'o.value']].rename(columns={'o.value': 'group'})

mapping_type.loc[mapping_type['value'].str.startswith("http://www.wikidata.org/entity"), "group"] = 'WIKIDATA'
mapping_type['group'] = mapping_type['group'].apply(lambda i: i.title().replace("_", " ") if str(i) != "nan" else i)
mapping_type.head()

,value,group
0,https://collection.sciencemuseumgroup.org.uk/p...,Person
1,http://www.wikidata.org/entity/Q3568968,Wikidata
2,https://collection.sciencemuseumgroup.org.uk/o...,Object
3,plastic,NaN
4,https://collection.sciencemuseumgroup.org.uk/p...,Organisation


## 3. By part of collection (SMG internal collection categories)

For this one we use SPARQL with the predicate `sdo:isPartOf`. We keep the `skos:hasTopConcept` value for any entity which doesn't have a `sdo:isPartOf` value (i.e. is not a collection object).

In [8]:
collection_query = """PREFIX sdo: <https://schema.org/>
SELECT DISTINCT ?s ?o WHERE {        
  ?s sdo:isPartOf ?o.
}
"""

collection_res = make_sparql_request(collection_query)

In [9]:
collection_res_df = pd.json_normalize(collection_res['results']['bindings'])[['s.value', 'o.value']]
mapping_collection_category = ent_idx_mapping.copy()

mapping_collection_category = pd.merge(
    left=mapping_collection_category, 
    right=collection_res_df, 
    left_on="value",
    right_on="s.value",
    how="left"
)[['value', 'o.value']].rename(columns={'o.value': 'group'})

mapping_collection_category['group'] = mapping_collection_category['group'].apply(lambda i: "Category - " + str(i) if str(i)!= "nan" else i)
mapping_collection_category['group'] = mapping_collection_category['group'].combine_first(mapping_type['group'])

mapping_collection_category.head()

,value,group
0,https://collection.sciencemuseumgroup.org.uk/p...,Person
1,http://www.wikidata.org/entity/Q3568968,Wikidata
2,https://collection.sciencemuseumgroup.org.uk/o...,Category - Therapeutics
3,plastic,NaN
4,https://collection.sciencemuseumgroup.org.uk/p...,Organisation


## 4. Export mappings

In [10]:
export_dir = "../data/processed/embedding_colour_mappings"

if not os.path.exists(export_dir): os.mkdir(export_dir)

In [11]:
def export_mapping(mapping_df, filename):
    mapping_df.to_csv(os.path.join(export_dir, filename), sep="\t", header=None)
    
export_mapping(mapping_database, 'mapping_database.tsv')
export_mapping(mapping_type, 'mapping_type.tsv')
export_mapping(mapping_collection_category, 'mapping_collection_category.tsv')